In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('host_phage_results_total.tsv', delimiter='\t')
data_first_copy, data_second_copy = data.copy(), data.copy()
data.head(5)

,id,name,width,sites,ic,re,llr,e_value,bayes_threshold,motif_scores,motif_probs,motif_reg_expr,command_line,reason_for_stopping,EPS,fasta
0,MEME_0_1115_motif_1,WHAYVHHAVHMWWYWAVHAAHWAHWHHTYYCYHWYM,36,110,14.6,12.4,947,1.800000e-28,8.4566,MEME_0_1115_motif_1.scores.tsv,MEME_0_1115_motif_1.probs.tsv,[ATC][TAC][AT][TCA][CAG][TAC][ACT][AT][AGC][AT...,meme host_phage_1/GCA_000023765.2_ASM2376v2_an...,Stopped because motif E-value > 1.00e-02.,MEME_0_1115_logo1.eps,GCA_000023765.2_ASM2376v2_and_phages_UPS.fasta
1,MEME_0_1115_motif_2,CTTTTWRAKGAGGRKYRTMSAYGRARYACSGWGRSS,36,6,47.0,51.0,212,1.900000e-16,11.8957,MEME_0_1115_motif_2.scores.tsv,MEME_0_1115_motif_2.probs.tsv,CTTTT[TA][GA]A[GT]G[AG]GG[AG][GT][CT][AG]T[AC]...,meme host_phage_1/GCA_000023765.2_ASM2376v2_an...,Stopped because motif E-value > 1.00e-02.,MEME_0_1115_logo2.eps,GCA_000023765.2_ASM2376v2_and_phages_UPS.fasta
2,MEME_0_1115_motif_3,CCAAGACTTGTTGTTAAACGCTTTAAAAAAGGAGAC,36,3,72.0,70.2,146,1.500000e-07,12.1073,MEME_0_1115_motif_3.scores.tsv,MEME_0_1115_motif_3.probs.tsv,CCAAGACTTGTTGTTAAACGCTTTAAAAAAGGAGAC,meme host_phage_1/GCA_000023765.2_ASM2376v2_an...,Stopped because motif E-value > 1.00e-02.,MEME_0_1115_logo3.eps,GCA_000023765.2_ASM2376v2_and_phages_UPS.fasta
3,MEME_0_1115_motif_4,TGHHHNAYAARGHWAAWRRHKATRAARDGAAAADA,35,23,24.5,21.6,345,8.600000e-11,10.6664,MEME_0_1115_motif_4.scores.tsv,MEME_0_1115_motif_4.probs.tsv,[TC]G[TAC][TAC][ACT][CAGT][AT][TC][AT][AT][AG]...,meme host_phage_1/GCA_000023765.2_ASM2376v2_an...,Stopped because motif E-value > 1.00e-02.,MEME_0_1115_logo4.eps,GCA_000023765.2_ASM2376v2_and_phages_UPS.fasta
4,MEME_0_1115_motif_5,GCGTTAGTAGATGCCCKATAACTTGTAATTTCCTAC,36,4,58.8,58.4,162,2.100000e-07,12.0333,MEME_0_1115_motif_5.scores.tsv,MEME_0_1115_motif_5.probs.tsv,GCGTTAG[TC]A[GA]A[TA]GC[CA][CT][GT]A[TA]A[AC][...,meme host_phage_1/GCA_000023765.2_ASM2376v2_an...,Stopped because motif E-value > 1.00e-02.,MEME_0_1115_logo5.eps,GCA_000023765.2_ASM2376v2_and_phages_UPS.fasta


[In this article](https://www.addgene.org/mol-bio-reference/promoters/)  -35 element: TTGACA

[From Wikipedia]('https://en.wikipedia.org/wiki/Promoter_(genetics)#Bacterial')  -35 element: TTGACA, -10 element: TATAAT

[In this article](https://www.cell.com/cell/fulltext/S0092-8674(11)01339-0) -10 element: TACAAT

[I took info from here](https://www.sciencedirect.com/topics/biochemistry-genetics-and-molecular-biology/pribnow-box) - useful link

[And also from here](https://watermark.silverchair.com/197-2-187.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAAt0wggLZBgkqhkiG9w0BBwagggLKMIICxgIBADCCAr8GCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMExbp9zzphpgcZa9rAgEQgIICkDAsloTW-RETW1DdWzGbaugRz42fu5uLmPtkN9kDYdICLaHiiVBSKCWIT-Uzkn7NW7tfsIYtmZv8DU2yzY9eL3PFTl_dmkntZBaZEACpDrTpwmEafYC5pJn9r1idgzc4fnOTdR683Ay02eXyMcLyNWv-l_lvMqZ58uhRAfbKXEfQkRfd6OO_NasPNAERlSdjSYHs7HUlvlzwzaG0Ei2u_NUA_50eVY7mGtEMcQFcGYHPI2dU-n8UAFSSyui0YzkmC8ETgblqOhtGALy1VopaGyCY_pB8uSQIsgR2U08KJt985cD9NfMBRHXzJa-JE0Wp_Vdwj_NXja-H_eRmkCzDs2A4oqwqBmBWZ0akd9YZdSWfwN_agYRhivvNBJ7nGGLJkyT8EN4ArwFY67ywulk3GGjWLddfJmNxqK-cuy0eqOghcke1WjJBKxlCV1lsTUx6jxfUe6aM36NTaWwmTcU1CS_FSb3PmYWTQ_2rQmByvNE3Rc1mb9LnC_MdpceW77EP5MM21yf58T8XJYHzrYoumPmeoRczYfRZnpFOr0whGewd3i0WpoNjL8bnXNAWmaL44A5ZQThYfIkv1-0aUCUG7wOET38kpAjoszBynai41Jec9iwBnAsoWTqGZP_yzv8ZdAFMhZq1-fH8T4LM46rf8dFKjEm0ZIs08JpZRY9Bb3CvTZOA55U4josXxwupdru5UNdCzvcqdw75mcZ1oOnv0eCAH_zuJmlugRqrXVtJSqsJ0c72c1irnuDFts8Z9VPfm2Hdx2u-JDtnKM8J0LkmAn5A9pBYwsgO7t-CTKzQBb_N8YfGtYDJTs_dtyQwBpZe1Ul8ek6SZgLIa0Rqy7ldneGkIY87u5DEv-y0jGiyDUKb) - also useful


## Ambiguous symbols 
from https://meme-suite.org/meme/doc/alphabet-format.html#standard_DNA

$$ U = T $$ 
$$ R = AG $$
$$ Y = CT $$
$$ K = GT $$
$$ M = AC $$
$$ S = CG $$
$$ W = AT $$
$$ B = CGT $$
$$ D = GAT $$
$$ H = ACT $$ 
$$ V = ACG $$
$$ N = ACGT $$ 
$$ X = ACGT $$ 

# Section a)

## First Way

Here, using the alphabet above, I matched all letters related to letters in -35 & -10 elements.

In [ ]:
# in you i forget some

# pattern_10.finditer(name[i]) - finds all matches
# pattern_10.findall(name[i]) - preint out all matches
# pattern_10.match(name[i]) - return first match in the begining in the string (only)
# pattern_10.search(name[i]) - search from the entire sentence

In [3]:
T = '[tuykwbdhnx]'
A = '[armwdhvnx]'
C = '[cymsbhvnx]'
G = '[grksbdvnx]'

pattern_35 = re.compile(T + T + G + A + C + A, re.I) # re.IGNORECASE - flag
pattern_10 = re.compile(T + A + '(' + T + '|c)' + A + A + T, re.I)

occurrences = []
names = data_first_copy.name

print("Length before removing: ", len(data_first_copy))

for i in names.index:
    if (pattern_10.search(names[i]) or pattern_35.search(names[i])):
        occurrences.append(i)
            
data_first_copy=data_first_copy.drop(occurrences)

print("Length after removing: ", len(data_first_copy))

Length before removing:  1946
Length after removing:  1344


## Second Way
Just deleting only those motifs that exactly match the ones described in the articles

In [4]:
pattern_35 = re.compile('ttgaca', re.I) # re.IGNORECASE - flag
pattern_10 = re.compile('ta[tc]aat', re.I)

occurrences = []
names = data_second_copy.name

print("Length before removing: ", len(data_second_copy))

for i in names.index:
    if (pattern_10.search(names[i]) or pattern_35.search(names[i])):
        occurrences.append(i)
            
data_second_copy=data_second_copy.drop(occurrences)

print("Length after removing: ", len(data_second_copy))

Length before removing:  1946
Length after removing:  1895


### Next, we will perform the same operations for both datasets, separating them as first and second in the variable names

# Section b)

In [12]:
# for the first copy

motif_positions_first = pd.DataFrame(index = data_first_copy.index,
                                     columns = ['name', 'regex', 'average_start',
                                                 'average_end', 'std_start', 'std_end'])

for index, name, regex, fasta_name in zip(data_first_copy.index,
                                      data_first_copy.name,
                                      data_first_copy.motif_reg_expr,
                                    data_first_copy.fasta):
    
    start_positions = np.array([])
    end_positions = np.array([])
    motif_name = name # initializing
    
    with open('host_and_phage_upstream_regions/' + fasta_name, 'r') as file:
        sequence = file.read() #reading sequence
        
    sequences = [seq.split('\n')[1] for seq in sequence[1:].split('>')]
    for seq in sequences:
        for match in re.compile(regex).finditer(seq): # finding matches in all sequences
            start_positions = np.append(start_positions,
                                        match.span()[0])
            end_positions = np.append(end_positions,
                                        match.span()[1])
    
    motif_positions_first.name[index] = name
    motif_positions_first.regex[index] = regex
    motif_positions_first.average_start[index] = start_positions.mean()
    motif_positions_first.average_end[index] = end_positions.mean()
    motif_positions_first.std_start[index] = start_positions.std()
    motif_positions_first.std_end[index] = end_positions.std() # adding all the info

<ipython-input-12-76bbad4ee05b>:29: RuntimeWarning: Mean of empty slice.
  motif_positions_first.average_start[index] = start_positions.mean()
<ipython-input-12-76bbad4ee05b>:30: RuntimeWarning: Mean of empty slice.
  motif_positions_first.average_end[index] = end_positions.mean()


In [13]:
# and for the second

motif_positions_second = pd.DataFrame(index = data_second_copy.index,
                                     columns = ['name', 'regex', 'average_start',
                                                 'average_end', 'std_start', 'std_end'])

for index, name, regex, fasta_name in zip(data_second_copy.index,
                                      data_second_copy.name,
                                      data_second_copy.motif_reg_expr,
                                    data_second_copy.fasta):
    
    start_positions = np.array([])
    end_positions = np.array([])
    motif_name = name # initializing for each motif specific arrays
    
    with open('host_and_phage_upstream_regions/' + fasta_name, 'r') as file:
        sequence = file.read() #reading sequence
        
    sequences = [seq.split('\n')[1] for seq in sequence[1:].split('>')]
    for seq in sequences:
        for match in re.compile(regex).finditer(seq): # finding matches in all sequences
            start_positions = np.append(start_positions,
                                        match.span()[0])
            end_positions = np.append(end_positions,
                                        match.span()[1])
    
    motif_positions_second.name[index] = name
    motif_positions_second.regex[index] = regex
    motif_positions_second.average_start[index] = start_positions.mean()
    motif_positions_second.average_end[index] = end_positions.mean()
    motif_positions_second.std_start[index] = start_positions.std()
    motif_positions_second.std_end[index] = end_positions.std() # adding all the info

<ipython-input-13-8483012c9f18>:29: RuntimeWarning: Mean of empty slice.
  motif_positions_second.average_start[index] = start_positions.mean()
<ipython-input-13-8483012c9f18>:30: RuntimeWarning: Mean of empty slice.
  motif_positions_second.average_end[index] = end_positions.mean()


Also, as you can see, I didn't find how to change a whole raw in one command - so I wrote changes for each position by hands. It would be cool if you tell me how to do this!

In [14]:
# Just creating more tsv files

# motif_positions_first.to_csv('motif_positions_first.tsv', sep='\t')
# motif_positions_second.to_csv('motif_positions_second.tsv', sep='\t')

# Section c)

In [79]:
# for the first way

location = []

for index, regex, fasta_name in zip(data_first_copy.index,
                                      data_first_copy.motif_reg_expr,
                                    data_first_copy.fasta):
    local = []
    with open('host_and_phage_upstream_regions/' + fasta_name, 'r') as file:
        sequence = file.read() #reading sequence
        
    sequences = [(seq.split('\n')[0], seq.split('\n')[1]) for seq in sequence[1:].split('>')]
    for name, seq in sequences:
        if re.compile(regex).search(seq): # search matches in all sequences
            local.append(name)
    
    local = set([x[:6] for x in local]) # I can check only 6 characters 
                                    #to check who the profile belongs to 
    
    if (local): # I also check this because in some cases local is an empty set
        if (local.__len__() > 1):
            location.append('BOTH')
            continue
    
        for z in local:
            location.append('HOST' if (z == sequences[0][0][:6]) else 'PHAGE')
    else:
        location.append('NOT FOUND') # and for these cases I set this value NOT FOUND

data_first_copy['location'] = location

In [78]:
# second one, the same work

location = []

for index, regex, fasta_name in zip(data_second_copy.index,
                                      data_second_copy.motif_reg_expr,
                                    data_second_copy.fasta):
    local = []
    with open('host_and_phage_upstream_regions/' + fasta_name, 'r') as file:
        sequence = file.read() #reading sequence
        
    sequences = [(seq.split('\n')[0], seq.split('\n')[1]) for seq in sequence[1:].split('>')]
    for name, seq in sequences:
        if re.compile(regex).search(seq): # search matches in all sequences
            local.append(name)
    
    local = set([x[:6] for x in local])
    
    if (local):
        if (local.__len__() > 1):
            location.append('BOTH')
            continue
    
        for z in local:
            location.append('HOST' if (z == sequences[0][0][:6]) else 'PHAGE')
    else:
        location.append('NOT FOUND')

data_second_copy['location'] = location

In [77]:
# whole tsv file

location = []

for index, regex, fasta_name in zip(data.index,
                                      data.motif_reg_expr,
                                    data.fasta):
    local = []
    with open('host_and_phage_upstream_regions/' + fasta_name, 'r') as file:
        sequence = file.read() #reading sequence
        
    sequences = [(seq.split('\n')[0], seq.split('\n')[1]) for seq in sequence[1:].split('>')]
    for name, seq in sequences:
        if re.compile(regex).search(seq): # search matches in all sequences
            local.append(name)
    
    local = set([x[:6] for x in local])
    
    if (local):
        if (local.__len__() > 1):
            location.append('BOTH')
            continue
    
        for z in local:
            location.append('HOST' if (z == sequences[0][0][:6]) else 'PHAGE')
    else:
        location.append('NOT FOUND')

data['location'] = location

In [80]:
# data.to_csv('host_phage_results_total.tsv', sep='\t')
# data_first_copy.to_csv('host_phage_results_total_first.tsv', sep='\t')
# # motif_positions_second.to_csv('host_phage_results_total_second.tsv', sep='\t')